# Zero-shot text classification with ollama and Llama 3.1 8B model

This notebook illustrates how to use the Llama 3.1 8B model via `ollama` for text classification.

<br>
<a target="_blank" href="https://colab.research.google.com/github/haukelicht/advanced_text_analysis/blob/main/notebooks/llm_zeroshot_classification_ollama.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

#### 🚨🚨 **_If_** your are on **Colab**, follow the next instructions:

1. go to the menu (in the top left of this window bleow the file name and right of the Colab icon "CO"),
2. select "Runtime",
3. click on "Change" runtime type
4. in the Hardware accelaration tab, choose "T4"
5. confirm you selection by clicking "Save"
6. aggree to starting a new runtim if asked

Now, run the next cell:

In [1]:
# check if on colab
COLAB = True
try:
    import google.colab
except:
    COLAB=False

if COLAB:
    # install required packages
    !pip install -q ollama==0.3.3 colab-xterm==0.2.0
    # activate xterm
    %load_ext colabxterm
    # install ollama and start the ollama server 
    !curl -fsSL https://ollama.com/install.sh | sh
    !ollama serve

if COLAB:
    # download custom utils (not needed)
    # !mkdir -p utils
    # !curl -o utils/io.py https://raw.githubusercontent.com/haukelicht/advanced_text_analysis/main/notebooks/utils/io.py
    pass

if COLAB:
    data_path = 'https://raw.githubusercontent.com/haukelicht/advanced_text_analysis/refs/heads/main/data/'
else:
    import os
    data_path = os.path.join('..', 'data', '')

#### 🚨🚨 **_If_** your are on **Colab**, follow the next instructions:

1. execute the next cell (which runs `%xterm`)
2. then run in the console/terminal that opens in the cell the following lines:

```bash
curl -fsSL https://ollama.com/install.sh | sh
ollama serve
```

In [2]:
# ONLY RUN THIS WHEN ON COLAB !!!
if COLAB:
    %xterm
    # next run: 
    #  curl -fsSL https://ollama.com/install.sh | sh
    #  ollama serve

Now continue: 

In [3]:
import os
from ollama import Client
import re

In [4]:
# create the ollama client that communicates with the `ollama` server running in the background
client = Client()
MODEL = 'llama3.1:8b' # currently the latest version of GPT-4o

In [5]:
get_available_models = lambda : [m['model'] for m in client.list()['models']]

if MODEL not in get_available_models():
  print(f"Model not found. Running `client.pull('{MODEL}')` to download it.")
  client.pull(MODEL)

# confirm
assert MODEL in get_available_models()

## Define the task

In this example, we adapt the instruction for one of the tweet classification tasks examined in Gilardi et al. ([2023](https://www.pnas.org/doi/10.1073/pnas.2305016120)) "ChatGPT outperforms crowd workers for text-annotation tasks"

- see [this README file](../data/labeled/gilardi_chatgpt_2023/README.md) for a description of the data and tasks covered in the paper
- see [this file](../data/labeled/gilardi_chatgpt_2023/instructions.md) for a copy of their original task instructions

In [6]:
instructions = """
For each tweet in the sample, follow these instructions:

1. Carefully read the text of the tweet, paying close attention to details.
2. Classify the tweet as either relevant (1) or irrelevant (0)
"""

categories = ["Relevant", "Irrelevant"]

defintions = """
Tweets should be coded as RELEVANT when they directly relate to content moderation, as defined above. This includes tweets that discuss: social media platforms’ content moderation rules and practices, governments’ regulation of online content moderation, and/or mild forms of content moderation like flagging.
Tweets should be coded as IRRELEVANT if they do not refer to content moderation, as defined above, or if they are themselves examples of moderated content. This would include, for example, a Tweet by Donald Trump that Twitter has labeled as “disputed”, a tweet claiming that something is false, or a tweet containing sensitive content. Such tweets might be subject to content moderation, but are not discussing content moderation. Therefore, they should be coded as irrelevant for our purposes.
"""

In [7]:
# Let's format the prompt
prompt = f"Classify the following text into one of the given categories: {categories}\n{defintions}\nOnly include the selected category in your response and no further text."
print(prompt)

Classify the following text into one of the given categories: ['Relevant', 'Irrelevant']

Tweets should be coded as RELEVANT when they directly relate to content moderation, as defined above. This includes tweets that discuss: social media platforms’ content moderation rules and practices, governments’ regulation of online content moderation, and/or mild forms of content moderation like flagging.
Tweets should be coded as IRRELEVANT if they do not refer to content moderation, as defined above, or if they are themselves examples of moderated content. This would include, for example, a Tweet by Donald Trump that Twitter has labeled as “disputed”, a tweet claiming that something is false, or a tweet containing sensitive content. Such tweets might be subject to content moderation, but are not discussing content moderation. Therefore, they should be coded as irrelevant for our purposes.

Only include the selected category in your response and no further text.


In [8]:
texts = [
    # negative examples ("irrelevant")
    "\"Turns out Mike Bloomberg is exactly what Elizabeth Warren needed to break through in the 2020 Democratic primary. And he’s not just a foil for her on the campaign trail — this is something she believes in, and it shows.\" https://t.co/1SyaHXrZlO",
    "@blackhat___05 ye raha new user name change kiya kamine ne😡🗡️😡🗡️😡🗡️😡 karo abhi FNfollow reopt aur block",
    "The Kid!\n \nRETWEET for a chance at a @RawlingsSports baseball signed by Ken Griffey Jr. and tune in to #Junior tonight at 8pm ET/5pm PT on MLB Network.\n \nRules: https://t.co/MdkXLh1CdN | NoPurNec, US 18+, Ends 6/22 https://t.co/8Xw0HpHz2G",
    "TW / gore \n\nif you come across an account and want to block them, make sure to cover the bottom half of your screen. the gore is normally at the bottom of the screen. again, stay safe, and take precaution",
    "@Godlesswh_re Blocked.  Is this another Nick account?",
    # positive examples ("relevant")
    "Twitter we want you to suspend Marcon's account.\n#twitterSuspendMacronAccount #TwitterSuspendMarcon @verified @Twitter @TwitterSupport",
    "Twitter needs to permanently suspend @realDonaldTrump account.  Who's with me?",
    "Toei is one of the most active reporters of content on Youtube and everything runs through an auto filter. Today, Toei dropped a ridiculous volume of their own series onto an official Youtube channel and GOT BANNED AND REPORTED BY THEMSELVES, TOEI.",
    "Marsha Blackburn: We Are Looking at Antitrust Laws and Section 230 on Tech Censorship https://t.co/lsOWzD0Yri",
    "#Facebook has banned the iconic photograph of a #Soviet solider waving the #USSR flag over the #Reichstag in May 1945. The social network claims the image violates its community guidelines for dangerous people and organizations...\n\nMORE: https://t.co/arpDN9Ss0P https://t.co/KGtGwE4D5J"
]

### A single text example

In [9]:
text_input = texts[0]

In [10]:
# construct to conversation history
messages = [
  # system prompt
  {"role": "system", "content": prompt},
  # user input
  {"role": "user", "content": text_input},
]

In [11]:
# set some options controlling generation behavior
opts = {
    'seed': 42,         # seed controlling random number generation and thus stochastic generation
    'temperature': 0.0, # hyper parameter controlling "craetivity", see https://learnprompting.org/docs/basics/configuration_hyperparameters
    'max_tokens': 3     # maximum numbers of tokens to generate in completion
}

In [12]:
# request a chat completion
response = client.chat(MODEL, messages, options=opts)

In [13]:
# parse the response
response['message']['content']

'Relevant'

In [14]:
response

{'model': 'llama3.1:8b',
 'created_at': '2024-09-25T19:44:49.28069Z',
 'message': {'role': 'assistant', 'content': 'Relevant'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1271586625,
 'load_duration': 72886000,
 'prompt_eval_count': 262,
 'prompt_eval_duration': 1087608000,
 'eval_count': 3,
 'eval_duration': 69776000}

### Iterate over multiple examples

Let's first define a custom function to classify tweets:

In [15]:
from typing import Dict, Optional
def classify_tweet(text, system_message, model, options: Optional[Dict]=None):

  # clean the text 
  text = re.sub(r'\s+', ' ', text).strip()

  # construct input

  messages = [
    # system prompt
    {"role": "system", "content": system_message},
    # user input
    {"role": "user", "content": text},
  ]
  if not options:
    options = {'seed': 42, 'temperature': 0.0, 'max_tokens': 3}
  response = client.chat(model, messages, options=options)
  
  result = response['message']['content']
  
  return result

Now we can iterate over example texts:

In [16]:
classifications = [classify_tweet(text, prompt, model=MODEL) for text in texts]
classifications

['Relevant',
 'Irrelevant',
 'Irrelevant',
 'Relevant',
 'Relevant',
 'Relevant',
 'Irrelevant',
 'Relevant',
 'Relevant',
 'Relevant']

- 3/5 negative examples classified correctly
- 3/5 positive examples classified correctly